In [19]:

import numpy as np
import pandas as pd
import plotly.graph_objects as go


#leer el archivo datos.xlsm y solo la hoja llamada "Datos"
df = pd.read_excel('datos.xlsm', sheet_name='Datos')
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Fecha                    268 non-null    datetime64[ns]
 1   Estudio                  268 non-null    object        
 2   Peso[kg]                 268 non-null    int64         
 3   Distancia foco-detector  268 non-null    object        
 4   Proyección               268 non-null    object        
 5   kV                       268 non-null    int64         
 6   mAs                      268 non-null    float64       
 7   Img Aprobadas            268 non-null    int64         
 8   Img Rechazadas           267 non-null    float64       
 9   Dosis [mGy]              268 non-null    float64       
 10  Dosis Total [mGy]        144 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 23.2+ KB


,Fecha,Estudio,Peso[kg],Distancia foco-detector,Proyección,kV,mAs,Img Aprobadas,Img Rechazadas,Dosis [mGy],Dosis Total [mGy]
0,2024-11-25,Pie,63,Bucky,AP-PA,57,4.5,1,0.0,0.09,NaN
1,2024-11-25,Pie,63,Bucky,LAT,54,4.5,1,0.0,0.08,"0,17"
2,2024-11-26,Pie,63,Bucky,AP-PA,57,4.5,1,0.0,0.09,NaN
3,2024-11-26,Pie,63,Bucky,LAT,54,4.5,1,0.0,0.08,"0,17"
4,2024-11-26,Tórax,56,Bucky,AP-PA,106,2.5,1,0.0,0.12,NaN


In [20]:
# --- 1) Nos quedamos solo con las filas donde 'Dosis Total [mGy]' no es NaN,es decir cada procedimiento unico
df_totales = df.dropna(subset=["Dosis Total [mGy]"])

# Convertir "Dosis Total [mGy]" a float, reemplazando la coma por punto
df_totales["Dosis Total [mGy]"] = df_totales["Dosis Total [mGy]"].str.replace(",", ".").astype(float)

# --- 2) (Opcional) Ordenar procedimientos por mediana para que el eje X quede más legible ---
orden = (df_totales
         .groupby("Estudio")["Dosis Total [mGy]"]
         .median()
         .sort_values()
         .index
         .tolist())

# --- 3) Graficar boxplots por Estudio ---
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(
    x=df_totales["Estudio"],      # categorías (un box por Estudio)
    y=df_totales["Dosis Total [mGy]"],        # valores
    name="Dosis Total [mGy]",
    boxpoints="outliers",               # muestra solo outliers (usa "all" para todos los puntos)
    boxmean=True                        # dibuja la media (usa "sd" para mostrar también ±1 SD)
))

fig.update_layout(
    title="Distribución de 'Dosis Total [mGy]' por procedimiento (boxplots)",
    xaxis_title="Estudio",
    yaxis_title="Dosis Total [mGy]",
)

# aplica el orden por mediana en el eje X (quita esta línea si no quieres reordenar)
fig.update_xaxes(categoryorder="array", categoryarray=orden)

fig.show()


C:\Users\oscar\AppData\Local\Temp\ipykernel_5336\1693692100.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
df_totales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, 1 to 267
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Fecha                    144 non-null    datetime64[ns]
 1   Estudio                  144 non-null    object        
 2   Peso[kg]                 144 non-null    int64         
 3   Distancia foco-detector  144 non-null    object        
 4   Proyección               144 non-null    object        
 5   kV                       144 non-null    int64         
 6   mAs                      144 non-null    float64       
 7   Img Aprobadas            144 non-null    int64         
 8   Img Rechazadas           143 non-null    float64       
 9   Dosis [mGy]              144 non-null    float64       
 10  Dosis Total [mGy]        141 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 13.5+ KB


In [24]:
df_totales["Dosis Total [mGy]"].describe()  

count    141.000000
mean       1.368085
std        2.197448
min        0.000000
25%        0.290000
50%        0.590000
75%        1.520000
max       18.770000
Name: Dosis Total [mGy], dtype: float64

In [22]:
#grafica de barras para saber la cantidad de veces que se repite cada procedimiento en df_totales
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_totales["Estudio"].value_counts().index,
    y=df_totales["Estudio"].value_counts().values,
    name="Cantidad de Procedimientos",
))

fig.update_layout(
    title="Cantidad de Procedimientos por Estudio",
    xaxis_title="Estudio",
    yaxis_title="Cantidad de Procedimientos",
)

fig.show()

In [23]:
#Conteo de procedimiento "pie" en df_totales sin grafica
conteo_procedimientos = df_totales["Estudio"].value_counts()
print(conteo_procedimientos)

Estudio
Tórax                 63
Rodilla               12
Pelvis Cadera         12
Hombro                10
Col Lumbar             7
Columna Panorámica     6
Pie                    5
Pierna                 4
Col Torácica           4
Abdomen                4
Cervical               4
Femur                  3
Cráneo Simple          2
Mano                   2
Tobillo                2
Muñeca                 1
Cabeza                 1
Húmero                 1
Huesos Cara            1
Name: count, dtype: int64
